In [1]:
import pandas as pd
import numpy as np

In [2]:
webExtract = pd.read_csv("data/nyc_inspection_data/WebExtract.txt").fillna(0)

/Users/yutongpang/.virtualenvs/sqlVen27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
webExtract.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462.0,7188924968,8,2014-03-03 00:00:00,D,10F,2.0,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225.0,7182875005,39,2014-07-01 00:00:00,F,06A,23.0,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22 00:00:00,D,10B,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234.0,7184443838,27,2014-05-29 00:00:00,D,08C,10.0,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22 00:00:00,D,02G,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000


In [4]:
webExtract["INSPDATE"] = pd.to_datetime(webExtract.INSPDATE)

In [5]:
#webExtract = webExtract.sort_values(by=["INSPDATE","SCORE"])

In [6]:
#webExtract = webExtract.groupby("CAMIS").last()

In [7]:
webExtract['CAMIS'] = webExtract.index

In [8]:
cuisinegroup = webExtract.groupby("CUISINECODE").agg({'CAMIS':{"number":lambda x: x.nunique()},"SCORE":{"mean":np.mean,"std":np.std}})

In [9]:
cuisinegroup.head()

SCORE              CAMIS
                   std       mean  number
CUISINECODE                              
0            19.847350  18.899281     139
1            14.715950  19.850163     307
2            19.608895  25.394779    1877
3            14.927886  19.829075  128372
4            10.788450  15.079268     656

In [10]:
cuisinegroup.columns = cuisinegroup.columns.droplevel(0)

In [11]:
cuisinegroup['std'] = cuisinegroup['std']/(cuisinegroup.number)**0.5

In [12]:
cuisinegroup = cuisinegroup.sort_values(by="mean")

In [13]:
cuisinegroup = cuisinegroup[cuisinegroup.number>100]

In [14]:
cuisinegroup.head()

,std,mean,number
CUISINECODE,,,
99,0.281050,11.051061,2781
42,0.972753,13.482143,168
75,0.411204,14.281145,594
33,0.446464,14.630000,300
29,0.142587,14.652194,6449


In [15]:
cuisine = pd.read_csv("data/nyc_inspection_data/Cuisine.txt")

In [16]:
cuisine.head()

,CUISINECODE,CODEDESC
0,2,African
1,3,American
2,5,Asian
3,15,Cajun
4,17,Caribbean


In [17]:
cuisinegroup["CUISINECODE"] = cuisinegroup.index

In [18]:
result = pd.merge(cuisinegroup, cuisine, on="CUISINECODE")

In [19]:
result[result.CODEDESC=="French"]

,std,mean,number,CUISINECODE,CODEDESC
40,0.17683,20.354805,7576,35,French


In [32]:
cui = map(lambda x1,x2,x3,x4: (x1,x2,x3,x4) ,result.CODEDESC, result['mean'], result['std'], result.number)

In [33]:
cui[11]

('Caf\xe9/Coffee/Tea', 15.994683295044306, 0.10258473259987819, 15235)

In [27]:
cui[11] = ('Cafe/Coffee/Tea', 8.2528334786399302, 0.17138327968402459, 1147)

In [28]:
cui

[('Other', 11.051060769507371, 0.2810499493532852, 2781),
 ('Hotdogs/Pretzels', 13.482142857142858, 0.97275302465918034, 168),
 ('Soups & Sandwiches', 14.281144781144782, 0.41120364280059418, 594),
 ('Ethiopian', 14.630000000000001, 0.44646424638473392, 300),
 ('Donuts', 14.652194138626143, 0.14258747410534775, 6449),
 ('Hotdogs', 14.71822033898305, 0.44592994747111159, 472),
 ('Ice Cream, Gelato, Yogurt, Ices',
  14.730250990752973,
  0.19301212210536678,
  3785),
 ('Armenian', 15.079268292682928, 0.42121820555556011, 656),
 ('Cajun', 15.548387096774194, 0.64999641239546091, 186),
 ('Egyptian', 15.662125340599456, 0.63248674594367726, 367),
 ('Sandwiches', 15.712999099369558, 0.14508022097045123, 6662),
 ('Cafe/Coffee/Tea', 8.25283347863993, 0.1713832796840246, 1147),
 ('Juice, Smoothies, Fruit Salads',
  16.003679175864605,
  0.23709334099999477,
  2718),
 ('Sandwiches/Salads/Mixed Buffet',
  16.340146917917597,
  0.22012966721872634,
  3131),
 ('Hamburgers', 16.577529232333504, 0.13

In [29]:
len(cuisinegroup)

75

In [30]:
import pickle

In [31]:
with open('cui.pickle', 'wb') as f:
    pickle.dump(cui, f)

In [35]:
'Caf\xe9/Coffee/Tea'.decode('iso-8859-1')

u'Caf\xe9/Coffee/Tea'